In [55]:
#Import denpendencies
import pandas as pd
import os
import numpy as np

In [2]:
#get filepath and read csv file
filepath_students=os.path.join('raw_data','students_complete.csv')
filepath_school=os.path.join('raw_data','schools_complete.csv')
#read student data and school into a dataframe file
students_df=pd.read_csv(filepath_students)
school_df=pd.read_csv(filepath_school)

# print(students_df.head())
# print(school_df.head())

In [3]:
#Code in this cell is for District Summary

#calculate the number of total schools
#and get the same result as the line below
num_schools=len(school_df)
#using len(students_df['school'].value_counts()) to calculate the number of schools

#calculate the number of total students
num_students=len(students_df)
#use school_df['size'].sum() gives the same number of students as well

#Total Budget
total_budget=school_df['budget'].sum()

#Average Math Score
ave_math=students_df['math_score'].mean()

#Average Reading Score
ave_reading=students_df['reading_score'].mean()

#% Passing Math
#I assume >=60 is passing 
math_passing_percentage=(students_df[(students_df['math_score']>=60)]['math_score'].count()/students_df['math_score'].count())*100
#using len(students_df.loc[students_df['math_score']>=60,:]) can also give number of students with math score>=60

#% Passing Reading
#I assume >=60 is passing 
reading_passing_percentage=(students_df[(students_df['reading_score']>=60)]['reading_score'].count()/students_df['reading_score'].count())*100
#using len(students_df.loc[students_df['reading_score']>=60,:]) can also give number of students with math score>=60

# Overall Passing Rate
overall_passing=(math_passing_percentage+reading_passing_percentage)/2

#creat the Disctric Summary dataframe
disc_sum=pd.DataFrame([[num_schools,num_students,total_budget,ave_math,ave_reading,\
                        math_passing_percentage,reading_passing_percentage,overall_passing]],\
                      columns=['Total Schools','Total Students','Total Budget','Average Math Score',\
                               'Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate'])

#Format the summary a little
disc_sum['Total Students']=disc_sum['Total Students'].map('{:,}'.format)
disc_sum['Total Budget']=disc_sum['Total Budget'].map('${:,}'.format)
disc_sum['Average Math Score']=disc_sum['Average Math Score'].map('{:.2f}'.format)
disc_sum['Average Reading Score']=disc_sum['Average Reading Score'].map('{:.2f}'.format)
disc_sum['% Passing Math']=disc_sum['% Passing Math'].map('{:.2f}%'.format)
disc_sum['% Passing Reading']=disc_sum['% Passing Reading'].map('{:.2f}%'.format)
disc_sum['% Overall Passing Rate']=disc_sum['% Overall Passing Rate'].map('{:.2f}%'.format)
disc_sum


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,81.88,92.45%,100.00%,96.22%


In [4]:
#code in this cell is to prepare school_df to make a School Summary
#set the index of school_df by school 'name' so it can be used for calculation
#and concation with other data series in the next cell
school_df=school_df.set_index('name')

In [5]:
#code in this cell is to prepare school_df to make a School Summary
#drop the column of school ID since it is not required for the final summary
school_ID_drop_df=school_df.drop(columns=['School ID'])

In [6]:
#code in this cell is to create School Summary

#School Name,School Type,Total Students,Total School Budget are all existing columns in school_ID_drop_df
#As I have figured out, the 'size' column in the school_ID_drop_df is actually the number of students in the school 
#because school_ID_drop_df['size'].sum() = len(students_df)
#to caluclate Per Student Budget and add per student budget as a column into school_ID_drop_df
school_ID_drop_df['Per Student Budget']=school_ID_drop_df['budget']/school_ID_drop_df['size']

#Now use data from students_df to calculate Average Math and reading score per school
#create a group object grouped by school
grouped_school=students_df.groupby(by='school')

#calculate the average math and reading score by school
ave_math_school=grouped_school['math_score'].mean()
ave_reading_school=grouped_school['reading_score'].mean()

#calculate the percentage of passing for math and reading for each school

#Two steps 
#Step1.count the number of students passing math or reading respectively for each school by doing the following:
#1a.from students_df select students with score>=60 for math and reading respectively and convert each into a dataframe
#1b.in the new dataframe groupby 'school' and use count to get the number of students passing math or reading
num_math_pass_school=pd.DataFrame(students_df.loc[students_df['math_score']>=60,:]).groupby(by='school')['math_score'].count()
num_reading_pass_school=pd.DataFrame(students_df.loc[students_df['reading_score']>=60,:]).groupby(by='school')['reading_score'].count()

#Step2.calculate the passing rate for math and reading for each school
#by dividing the num_math_pass_school by total students in the school 
#note I use two data series from two dataframe for this division
#I can do the calculation becaues the two data series have the same index(school names)
math_pass_rate_school=(num_math_pass_school/school_ID_drop_df['size'])*100
reading_pass_rate_school=(num_reading_pass_school/school_ID_drop_df['size'])*100

#Concate the data series of'% Passing Math'and'% Passing Reading' with school_ID_drop_df
#name this df as school_sum
school_sum=pd.concat([school_ID_drop_df,ave_math_school,ave_reading_school,math_pass_rate_school,reading_pass_rate_school],axis=1)

#change the column names in the school_sum
school_sum.columns=['Type','Size','Budget','Per Student Budget','Average Math Score',\
                    'Average Reading Score','% Passing Math','% Passing Reading']
#add a column of Overall Passing Rate
school_sum['% Overall Passing Rate']=(school_sum['% Passing Math']+school_sum['% Passing Reading'])/2

#format columns of 'Budget' and 'Per Student Budget' since I am not going to do further calculation on it
school_sum['Budget']=school_sum['Budget'].map('${:,}'.format)
school_sum['Per Student Budget']=school_sum['Per Student Budget'].map('${:.2f}'.format)

school_sum


,Type,Size,Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,"$3,124,928",$628.00,77.048432,81.033963,89.529743,100.0,94.764871
Cabrera High School,Charter,1858,"$1,081,356",$582.00,83.061895,83.975780,100.000000,100.0,100.000000
Figueroa High School,District,2949,"$1,884,411",$639.00,76.711767,81.158020,88.436758,100.0,94.218379
Ford High School,District,2739,"$1,763,916",$644.00,77.102592,80.746258,89.302665,100.0,94.651333
Griffin High School,Charter,1468,"$917,500",$625.00,83.351499,83.816757,100.000000,100.0,100.000000
Hernandez High School,District,4635,"$3,022,020",$652.00,77.289752,80.934412,89.083064,100.0,94.541532
Holden High School,Charter,427,"$248,087",$581.00,83.803279,83.814988,100.000000,100.0,100.000000
Huang High School,District,2917,"$1,910,635",$655.00,76.629414,81.182722,88.858416,100.0,94.429208
Johnson High School,District,4761,"$3,094,650",$650.00,77.072464,80.966394,89.182945,100.0,94.591472
Pena High School,Charter,962,"$585,858",$609.00,83.839917,84.044699,100.000000,100.0,100.000000


In [7]:
#Code in the cell is to create Top Performing Schools (By Overall Passing Rate)
top_schools=school_sum.sort_values(by=['% Overall Passing Rate'],ascending=False).head()
top_schools

,Type,Size,Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356",$582.00,83.061895,83.975780,100.0,100.0,100.0
Griffin High School,Charter,1468,"$917,500",$625.00,83.351499,83.816757,100.0,100.0,100.0
Holden High School,Charter,427,"$248,087",$581.00,83.803279,83.814988,100.0,100.0,100.0
Pena High School,Charter,962,"$585,858",$609.00,83.839917,84.044699,100.0,100.0,100.0
Shelton High School,Charter,1761,"$1,056,600",$600.00,83.359455,83.725724,100.0,100.0,100.0


In [8]:
#Code in the cell is to create bottome performing schools(By overall Passing Rate)
bottom_schools=school_sum.sort_values(by='% Overall Passing Rate',ascending=True).head()
bottom_schools

,Type,Size,Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Figueroa High School,District,2949,"$1,884,411",$639.00,76.711767,81.158020,88.436758,100.0,94.218379
Rodriguez High School,District,3999,"$2,547,363",$637.00,76.842711,80.744686,88.547137,100.0,94.273568
Huang High School,District,2917,"$1,910,635",$655.00,76.629414,81.182722,88.858416,100.0,94.429208
Hernandez High School,District,4635,"$3,022,020",$652.00,77.289752,80.934412,89.083064,100.0,94.541532
Johnson High School,District,4761,"$3,094,650",$650.00,77.072464,80.966394,89.182945,100.0,94.591472


In [14]:
students_df.groupby(['grade','school'])['math_score'].mean().unstack(level='grade')

grade,10th,11th,12th,9th
school,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


In [50]:
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [58]:
students_df.head()
table =pd.pivot_table(students_df, values='math_score', index=['school'], columns=['grade'], aggfunc=np.mean)
table

grade,10th,11th,12th,9th
school,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


In [49]:
df.pivot_table(index=col1,values=[col2,col3],aggfunc=mean)

grade,10th,11th,12th,9th
school,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


In [217]:
#code in this cell is to create a table for Math Scores by Grade for each school
#Group students_df by school and grade and get the average math score for each school
ave_math_school_grade=students_df.groupby(['grade','school'])['math_score'].mean()

#subset the average score of math of all schools for each grade 
#and concate them into one dataframe
math_scores_grade_sum=pd.concat([ave_math_school_grade.xs('9th',level='grade'),\
                                 ave_math_school_grade.xs('10th',level='grade'),\
                                ave_math_school_grade.xs('11th',level='grade'),\
                                 ave_math_school_grade.xs('12th',level='grade')],axis=1)
#change the column names
math_scores_grade_sum.columns=['Math 9th','Math 10th','Math 11th','Math 12th']
math_scores_grade_sum

,Math 9th,Math 10th,Math 11th,Math 12th
school,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [218]:
#code in this cell is to create a table for Math Scores by Grade for each school
#Group students_df by school and grade and get the average math score for each school
ave_reading_school_grade=students_df.groupby(['grade','school'])['reading_score'].mean()

#subset the average score of math of all schools for each grade 
#and concate them into one dataframe
reading_scores_grade_sum=pd.concat([ave_reading_school_grade.xs('9th',level='grade'),\
                                 ave_reading_school_grade.xs('10th',level='grade'),\
                                ave_reading_school_grade.xs('11th',level='grade'),\
                                 ave_reading_school_grade.xs('12th',level='grade')],axis=1)
#change the column names
reading_scores_grade_sum.columns=['Reading 9th','Reading 10th','Reading 11th','Reading 12th']
reading_scores_grade_sum

,Reading 9th,Reading 10th,Reading 11th,Reading 12th
school,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [219]:
#Code below is to create a table for Scores by School Spending

#create a dataframe with the columns of personal budget,Average Math Score, Average Reading Score,
#% Passing Math,% Passing Reading,Overall Passing Rate ,with an index of school name
#I have set the school_df index as school name already so I just need to concat all the columns
score_by_spending=pd.concat([school_ID_drop_df['Per Student Budget'],\
                             school_sum.iloc[:,[4,5,6,7,8]]],axis=1)

#bin and assign names since there is no specific way I am using qcut and break it by quaters
score_by_spending['Spending Range per Student']=pd.qcut(score_by_spending['Per Student Budget'],4,\
                                                        labels=['<$591.5','$591.5-628','$628-641.5','$641.5-655'])
#groupby the Spending Range per Student and make a dataframe 
spend_range_score_sum=pd.DataFrame(score_by_spending.groupby(['Spending Range per Student']).mean())

spend_range_score_sum




,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Range per Student,,,,,,
<$591.5,581.00,83.455399,83.933814,100.000000,100.0,100.000000
$591.5-628,615.50,81.899826,83.155286,97.382436,100.0,98.691218
$628-641.5,638.00,78.990942,81.917212,92.327965,100.0,96.163983
$641.5-655,650.25,77.023555,80.957446,89.106772,100.0,94.553386


In [220]:
#Code below is to create a table for Scores by School Size
#create a dataframe with the columns of school size,Average Math Score, Average Reading Score,
#% Passing Math,% Passing Reading,Overall Passing Rate ,with an index of school name
#I have set the school_df index as school name already so I just need to concat all the columns

score_by_size=pd.concat([school_ID_drop_df['size'],\
                             school_sum.iloc[:,[4,5,6,7,8]]],axis=1)

#bin and assign names since there is no specific way I am using qcut and break it by quaters
score_by_size['School Size']=pd.qcut(score_by_size['size'],3,labels=['Small (<1787)','Medium (1787-2927)','Big (2928-4976)'])

#drop 'size' column and groupby School Size and make it into a dataframe
score_by_size_sum=pd.DataFrame(score_by_size.drop(columns=['size']).groupby('School Size').mean())

score_by_size_sum

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1787),83.554500,83.850220,100.000000,100.0,100.000000
Medium (1787-2927),80.750065,82.769850,95.632216,100.0,97.816108
Big (2928-4976),76.993025,80.967495,88.955929,100.0,94.477965


In [221]:
#Code below is to create a table for Scores by school type
#create a dataframe with the columns of school type,Average Math Score, Average Reading Score,
#% Passing Math,% Passing Reading,Overall Passing Rate ,with an index of school name
#I have set the school_df index as school name already so I just need to concat all the columns
score_by_type=pd.concat([school_ID_drop_df['type'],\
                             school_sum.iloc[:,[4,5,6,7,8]]],axis=1)
#groupby the school type and make a dataframe 
score_by_type_sum=pd.DataFrame(score_by_type.groupby(['type']).mean())

score_by_type_sum

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.473852,83.896421,100.000000,100.0,100.000000
District,76.956733,80.966636,88.991533,100.0,94.495766
